Given a Coefs_ab file that represents the probability distribution of the a and b parameteres, we use this function to sample from it

In [36]:
import numpy as np
import scipy as sc
Coefs_ab = np.load('../Coefs_17-17-8_(0.1-0.1-0.01).npy')

In [37]:
'''
Makes the knots for the given Coefficients
Provided a range of valid values for each of the three parameters
Params:
    a_reg - number of valid regions in the a dimension
    b_reg - number of valid regions in the b dimension
    E_reg - number of valid regions in the E dimension
    a_min - minumum value in the a dimension, [Default = 0]
    a_max - maximum value in the a dimension, [Default = 1]
    b_min - minumum value in the b dimension, [Default = 0]
    b_max - maximum value in the b dimension, [Default = 1]
    E_min - minumum value in the E dimension, [Default = 1]
    E_max - maximum value in the E dimension, [Default = 6]
            Note that returned knot values will extend past these values for b-spline fitting purposes
Return:
    a_k - 1d array of knots for the a dimension
    b_k - 1d array of knots for the b dimension
    E_k - 1d array of knots for the E dimension
'''
def make_knots(a_reg,b_reg,E_reg,a_min=0,a_max=1,b_min=0,b_max=1,E_min=1,E_max=6):
    c_a = a_reg + 3
    c_b = b_reg + 3
    c_E = E_reg + 3
    ## Define the knots
    a_k = np.linspace(a_min,a_max,c_a - 3 + 1)
    b_k = np.linspace(b_min,b_max,c_b - 3 + 1)
    E_k = np.linspace(E_min,E_max,c_E - 3 + 1)
    ## add knot values on above and below the range of interest
    a_k = sc.interpolate.interp1d(np.arange(c_a - 3 + 1),a_k,bounds_error=False,fill_value='extrapolate')(np.arange(-3,c_a + 1))
    b_k = sc.interpolate.interp1d(np.arange(c_b - 3 + 1),b_k,bounds_error=False,fill_value='extrapolate')(np.arange(-3,c_b + 1))
    E_k = sc.interpolate.interp1d(np.arange(c_E - 3 + 1),E_k,bounds_error=False,fill_value='extrapolate')(np.arange(-3,c_E + 1))
    return a_k,b_k,E_k
'''
Given a 6d array of coefficients, evaluate the spline at the given array of points
Params:
    a,b,E - input parameter values
    Coefs - Coefs[i,j,k,q,r,s] is the coefficient on the a**q b**r E**s term in the i-th a region, j-th b region, and k-th E region
    knots: tuple (a_k,b_k,E_k) where each element is the 1d array of the knots defining the spline regions along each dimension
           if not supplied, make_knots is called with the default values.
Returns:
    Result of evaluating BSpline at the provided a,b,E values
'''
def Eval_from_Coefs(a,b,E,Coefs,knots=None):
    ## if knots aren't specified generate them from default values
    if knots == None:
        knots = make_knots(Coefs.shape[0],Coefs.shape[1],Coefs.shape[2])
    a_k = knots[0]
    b_k = knots[1]
    E_k = knots[2]
    a_i = np.searchsorted(a_k[3:-3], a, side='right')
    b_i = np.searchsorted(b_k[3:-3], b, side='right')
    E_i = np.searchsorted(E_k[3:-3], E, side='right')

    a_i -= (a_i > Coefs.shape[0]) # so that things don't break at the upper boundaries
    b_i -= (b_i > Coefs.shape[1])
    E_i -= (E_i > Coefs.shape[2])
    
    Z = 0
    for l in range(4):
        for m in range(4):
            for n in range(4):
                Z += Coefs[a_i-1,b_i-1,E_i-1,l,m,n] * a**l * b**m * E**n
    return np.exp(Z)

'''
Integrate each region of the spline at a specified energy and return a grid of the integral in each region
Params:
    Coefs: Coefs[i,j,k,q,r,s] is the coefficient on the a**q b**r E**s term in the intersection of the 
           i-th a region, j-th b region, and k-th E region
    knots: tuple (a_k,b_k,E_k) where each element is the 1d array of the knots defining the spline regions along each dimension
           if not supplied, make_knots is called with the default values.
    num_quad_nodes: number of nodes used for guassian quadrature, [Default = 7]
Returns:
    result[i,j,k] is the integral over the intersection of the i-th a region, j-th b region, and k-th E region
'''
def integrate_grid(Coefs,E,knots=None,num_quad_nodes=7):
    ## if knots aren't specified generate them from default values
    if knots == None:
        knots = make_knots(Coefs.shape[0],Coefs.shape[1],Coefs.shape[2])
    a_k = knots[0]
    b_k = knots[1]
    E_k = knots[2]
    
    ## create nodes for gaussian quadrature
    nodes_1d, weights_1d = np.polynomial.legendre.leggauss(num_quad_nodes)
    weights = np.tile(weights_1d,(num_quad_nodes,1)) * np.tile(weights_1d,(num_quad_nodes,1)).T
    nodes = np.tile(nodes_1d,(num_quad_nodes,1))
    nodes_array = np.tile(nodes.reshape(1,1,num_quad_nodes,num_quad_nodes),(Coefs.shape[0],Coefs.shape[1],1,1))
    nodesT_array = np.tile(nodes.T.reshape(1,1,num_quad_nodes,num_quad_nodes),(Coefs.shape[0],Coefs.shape[1],1,1))
    weights_array = np.tile(weights.reshape(1,1,num_quad_nodes,num_quad_nodes),(Coefs.shape[0],Coefs.shape[1],1,1))
    E_i = np.searchsorted(E_k[3:-3],E,side='right')
    E_i -= (E_i > Coefs.shape[2])

    Z = np.zeros((*Coefs.shape[:2],num_quad_nodes,num_quad_nodes))
    ## Z[grid of regions a][grid of regions b][grid of nodes a][grid of nodes b]

    l_a = np.tile(a_k[3:-4].reshape(-1,1,1,1),(1,Coefs.shape[1],num_quad_nodes,num_quad_nodes))
    h_a = np.tile(a_k[4:-3].reshape(-1,1,1,1),(1,Coefs.shape[1],num_quad_nodes,num_quad_nodes))
    l_b = np.tile(b_k[3:-4].reshape(1,-1,1,1),(Coefs.shape[0],1,num_quad_nodes,num_quad_nodes))
    h_b = np.tile(b_k[4:-3].reshape(1,-1,1,1),(Coefs.shape[0],1,num_quad_nodes,num_quad_nodes))
    
    for l in range(4):
        for m in range(4):
            for n in range(4):
                Z += np.tile(Coefs[:,:,E_i-1,l,m,n].reshape((Coefs.shape[0],Coefs.shape[1],1,1)),(1,1,num_quad_nodes,num_quad_nodes)) \
                        * (nodes_array*(h_a-l_a)/2 + (l_a+h_a)/2)**l * (nodesT_array*(h_b-l_b)/2 + (l_b+h_b)/2)**m * E**n
    return np.sum(np.exp(Z) * weights * (a_k[1]-a_k[0])*(b_k[1]-b_k[0])/4,axis=(2,3))

'''
perform a likelihood_test on the provided sample using the given Coefs
Params:
    a_sample: array of a values in the test sample
    b_sample: array of b values in the test sample
    E: log energy value at which we are testing
    Coefs: coefficients defining the model we are testing
           Coefs[i,j,k,q,r,s] is the coefficient on the (a**q b**r E**s) term in the intersection of the 
           i-th a region, j-th b region, and k-th E region
    knots: tuple (a_k,b_k,E_k) where each element is a 1d array of the knots defining the spline regions along each dimension
           if not supplied, make_knots is called with the default values.
Returns:
    log-likelihood of the given sample
'''
def likelihood_test(a_sample,b_sample,E,Coefs,knots=None):
    ## If knots aren't specified, generate them from default values
    if knots == None:
        knots = make_knots(Coefs.shape[0],Coefs.shape[1],Coefs.shape[2])
    norm_factor = integrate_grid(Coefs,E,knots).sum()
    return np.log(Eval_from_Coefs(a_sample,b_sample,E,Coefs,knots)).sum() - (np.size(a_sample) * np.log(norm_factor))

In [130]:
'''
Creates samples from the spline specified by Coefs, at a specified energy E
Required Parameters:
    Coefs: Coefficients that represent the spline
    E: Energy value (GeV) at which we are taking samples
    num_samples: The number of samples to take
Optional Parameters:
    knots: tuple (a_k,b_k,E_k) where each element is a 1d array of the knots defining the spline regions along each dimension
           if not supplied, make_knots is called with the default values.
    sample_depth: Number of times to divide regions during binary grid sampling. A sample_depth
                  of n will give a sample precision of of 1/2^n times the size of each spline region
                  [Default: 10]
    binning_offset: Determines whether to random offset to reduce binning errors [Default: True]
    num_quad_nodes: Number of nodes used for gaussian quadrature [Default: 7]
    raw_output: Determines whether to output raw values from the spline, or their transformed values [Default: False]
    ga_inv: Inverse transform function for the a parameter [Default: a**2]
    gb_inv: Inverse transform function for the b parameter [Defaule sqrt((1/b) - 1)]
'''
def sample_ab(Coefs, E, num_samples,
           knots=None, sample_depth=7,binning_offset=True,
           num_quad_nodes=7,raw_output=False,ga_inv=lambda a: a**-2,gb_inv=lambda b: np.sqrt(b**-1 - 1)):
    ## If knots aren't specified, generate them from default values
    if knots == None:
        knots = make_knots(Coefs.shape[0],Coefs.shape[1],Coefs.shape[2])
    a_k = knots[0]
    b_k = knots[1]
    E_k = knots[2]
    
    ## Create nodes and weights for 2d gaussian quadrature
    nodes_1d, weights_1d = np.polynomial.legendre.leggauss(num_quad_nodes)
    weights = np.tile(weights_1d,(num_quad_nodes,1)) * np.tile(weights_1d,(num_quad_nodes,1)).T

    E = np.log10(E)
    
    ## Subrouting to integrate the spline within a specified region
    def integrate(l_a,h_a,l_b,h_b,Coefs_abE):
        nodes_ = nodes_1d.reshape(1,num_quad_nodes,1)
        nodesT_ = nodes_1d.reshape(1,1,num_quad_nodes)
        Z = np.zeros([Coefs_abE.shape[0],num_quad_nodes,num_quad_nodes])
        for l in range(4):
            for m in range(4): 
                Z += Coefs_abE[:,:,:,l,m] * \
                            np.tile((nodes_*(h_a-l_a)/2 + (l_a+h_a)/2)**l,(1,1,num_quad_nodes)) * \
                            np.tile((nodesT_*(h_b-l_b)/2 + (l_b+h_b)/2)**m,(1,num_quad_nodes,1))
        return np.sum(np.exp(Z) * weights.reshape(1,num_quad_nodes,num_quad_nodes), axis=(-2,-1),keepdims=True) * (h_a-l_a)*(h_b-l_b)/4
    E_i = np.searchsorted(E_k[3:-3],E,side='right')
    E_i -= (E_i > Coefs.shape[2])
    CoefsE = (Coefs[:,:,E_i-1,:,:,:] * np.array([1,E,E**2,E**3]).reshape(1,1,1,1,4)).sum(axis = -1)
    
    ## Generate the indices for the regions by integrating over every region and randomly selecting based on their total probability
    nodes = np.tile(nodes_1d,(num_quad_nodes,1))
    nodes_array = np.tile(nodes.reshape(1,1,num_quad_nodes,num_quad_nodes),(CoefsE.shape[0],CoefsE.shape[1],1,1))
    nodesT_array = np.tile(nodes.T.reshape(1,1,num_quad_nodes,num_quad_nodes),(CoefsE.shape[0],CoefsE.shape[1],1,1))
    weights_array = np.tile(weights.reshape(1,1,num_quad_nodes,num_quad_nodes),(CoefsE.shape[0],CoefsE.shape[1],1,1))

    Z = np.zeros((*CoefsE.shape[:2],num_quad_nodes,num_quad_nodes))
    
    ## Z[grid of regions a][grid of regions b][grid of nodes a][grid of nodes b]
    l_a = np.tile(a_k[3:-4].reshape(-1,1,1,1),(1,CoefsE.shape[1],num_quad_nodes,num_quad_nodes))
    h_a = np.tile(a_k[4:-3].reshape(-1,1,1,1),(1,CoefsE.shape[1],num_quad_nodes,num_quad_nodes))
    l_b = np.tile(b_k[3:-4].reshape(1,-1,1,1),(CoefsE.shape[0],1,num_quad_nodes,num_quad_nodes))
    h_b = np.tile(b_k[4:-3].reshape(1,-1,1,1),(CoefsE.shape[0],1,num_quad_nodes,num_quad_nodes))  
    
    for l in range(4):
        for m in range(4):
            Z += np.tile(CoefsE[:,:,l,m].reshape(CoefsE.shape[0],CoefsE.shape[1],1,1),(1,1,num_quad_nodes,num_quad_nodes)) \
                    * (nodes_array*(h_a-l_a)/2 + (l_a+h_a)/2)**l * (nodesT_array*(h_b-l_b)/2 + (l_b+h_b)/2)**m
    integrated_grid = np.sum(np.exp(Z) * weights * (a_k[1]-a_k[0])*(b_k[1]-b_k[0])/4, axis=(2,3))
    
    ranges = np.insert(integrated_grid.reshape(-1).cumsum(),0,0)
    indices = np.searchsorted(ranges,np.random.rand(num_samples)*ranges[-1]) - 1
    a_regions = indices // CoefsE.shape[1]
    b_regions = indices % CoefsE.shape[1]

    
    Coefs_abE = CoefsE[a_regions,b_regions,:,:].reshape(num_samples,1,1,4,4)
    
    l_a = np.reshape(a_k[a_regions + 3],(num_samples,1,1))
    h_a = np.reshape(a_k[a_regions + 4],(num_samples,1,1))
    l_b = np.reshape(b_k[b_regions + 3],(num_samples,1,1))
    h_b = np.reshape(b_k[b_regions + 4],(num_samples,1,1))
    
    for division in range(sample_depth):
        m_a = (l_a + h_a)/2
        # Integrate left and right of m_a
        Z_left = integrate(l_a,m_a,l_b,h_b,Coefs_abE)
        Z_right = integrate(m_a,h_a,l_b,h_b,Coefs_abE)
        choose_left = np.random.rand(num_samples,1,1) < (Z_left / (Z_left + Z_right))
        l_a = np.where(choose_left,l_a,m_a)
        h_a = np.where(choose_left,m_a,h_a)

        m_b = (l_b + h_b)/2
        # Integrate above and bellow
        Z_bottom = integrate(l_a,h_a,l_b,m_b,Coefs_abE)
        Z_top = integrate(l_a,h_a,m_b,h_b,Coefs_abE)
        choose_bottom = np.random.rand(num_samples,1,1) < (Z_bottom / (Z_bottom + Z_top))
        l_b = np.where(choose_bottom,l_b,m_b)
        h_b = np.where(choose_bottom,m_b,h_b)
    if binning_offset:
        res_a = l_a + np.random.rand(num_samples,1,1)*(a_k[1]-a_k[0])/2**sample_depth
        res_b = l_b + np.random.rand(num_samples,1,1)*(b_k[1]-b_k[0])/2**sample_depth
    else:
        res_a = (l_a + h_a)/2
        res_b = (l_b + h_b)/2
    if raw_output:
        return res_a.reshape(-1),res_b.reshape(-1)
    else:
        return np.vectorize(ga_inv)(res_a.reshape(-1)),np.vectorize(gb_inv)(res_b.reshape(-1))